# LAB1 RNNs : Lucía Chicharro, María Montero y Patricia Ramos


Haremos una **predicción de temperaturas** de **24 horas en el futuro**, dada una **serie temporal** de **medidas de cantidades** (cada hora) coeespondientes a **presión** y **humedad** recogidas en el **pasado reciente** por un conjunto de sensores en lo alto de un edificio.

El dataset se ha obtenido de la estación meteorilógica del Max Plank Institure for Biogeochemistry en Jena, Alemania. [http://www.bgc-jena.mpg.de/wetter](http://www.bgc-jena.mpg.de/wetter)

El dataset se compone de 14 magnitudes, como temperatura, presión, humedad, dirección del viento, etc. grabadas cada 10' durante varios años (2009-2016). Descargamos y descomprimimos el dataset

In [1]:
!wget https://s3.amazonaws.com/keras-datasets/jena_climate_2009_2016.csv.zip
!unzip jena_climate_2009_2016.csv.zip

--2024-02-27 20:28:53--  https://s3.amazonaws.com/keras-datasets/jena_climate_2009_2016.csv.zip
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.111.126, 16.182.103.96, 52.217.121.192, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.111.126|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13565642 (13M) [application/zip]
Saving to: ‘jena_climate_2009_2016.csv.zip’

jena_climate_2009_2 100%[===================>]  12.94M  47.5MB/s    in 0.3s    

2024-02-27 20:28:53 (47.5 MB/s) - ‘jena_climate_2009_2016.csv.zip’ saved [13565642/13565642]

Archive:  jena_climate_2009_2016.csv.zip
  inflating: jena_climate_2009_2016.csv  
  inflating: __MACOSX/._jena_climate_2009_2016.csv  


**Echamos un vistazo al dataset**

In [2]:
import os
fname = os.path.join("jena_climate_2009_2016.csv")

with open(fname) as f:
    data = f.read()

lines = data.split("\n")
header = lines[0].split(",")
lines = lines[1:]
print(header)
print(len(lines))

['"Date Time"', '"p (mbar)"', '"T (degC)"', '"Tpot (K)"', '"Tdew (degC)"', '"rh (%)"', '"VPmax (mbar)"', '"VPact (mbar)"', '"VPdef (mbar)"', '"sh (g/kg)"', '"H2OC (mmol/mol)"', '"rho (g/m**3)"', '"wv (m/s)"', '"max. wv (m/s)"', '"wd (deg)"']
420451


Esto devuelve una cuenta de 420.451 líneas de datos (cada línea es un timestep: un registro de una fecha con 14 valores relativos a las condiciones meteorológicas)

**Parseamos los datos**

In [3]:
import numpy as np
temperature = np.zeros((len(lines),))
raw_data = np.zeros((len(lines), len(header) - 1))
for i, line in enumerate(lines):
    values = [float(x) for x in line.split(",")[1:]]
    # Guardamos columna 1 en el array temperatura
    temperature[i] = values[1]
    # Guardamos todas las columnas (temperatura
    # incluida) en el array raw_data
    raw_data[i, :] = values[:]

Ahora, **convertimos** las 420.551 **líneas de datos** en **arrays NumPy**: **un array para** la **temperatura** (en Celsius), y **otro para el resto de los datos**-las características que usaremos para predecir futuras temperaturas. Notar que **hemos descartado** la columna **"Date Time"**

**Computamos el número de muestras que usaremos para cada lote de datos**

Usaremos el 50% para entrenar, el siguiente 25% para validar y el último 25% para las pruebas.

In [4]:
num_train_samples = int(0.5 * len(raw_data))
num_val_samples = int(0.25 * len(raw_data))
num_test_samples = len(raw_data) - num_train_samples - num_val_samples
print("num_train_samples:", num_train_samples)
print("num_val_samples:", num_val_samples)
print("num_test_samples:", num_test_samples)

num_train_samples: 210225
num_val_samples: 105112
num_test_samples: 105114


### Preparación de los datos

**Normalizando los datos**

Dado que los datos están en escalas diferentes, normalizamos cada serie para que todas tomen valores pequeños en una escala similar.

In [5]:
mean = raw_data[:num_train_samples].mean(axis=0)
raw_data -= mean
std = raw_data[:num_train_samples].std(axis=0)
raw_data /= std

A continuación, **creamos un objeto ``Dataset``** que **produzca lotes** de datos de los **últimos cinco días** **junto con** una **temperatura objetivo** **dentro de las 24 horas siguientes**. Debido a que las **muestras** en el conjunto de datos son **altamente redundantes** (la muestra ``N`` y la muestra ``N + 1`` tendrán la mayoría de sus intervalos de tiempo en común), sería un desperdicio asignar memoria explícitamente para cada muestra. En su lugar, **generaremos las muestras sobre la marcha** y solo mantendremos en la memoria los arrays ``raw_data`` y de ``temperature`` originales, y nada más.

Podríamos escribir fácilmente un generador de Python para hacer esto, pero hay una utilidad incorporada en Keras que hace exactamente eso (**``timeseries_dataset_from_array()``**), por lo que podemos ahorrarnos algo de trabajo al usarlo. Por lo general, se puede usar para cualquier tipo de tarea de pronóstico de series temporales.

Usaremos ``timeseries_dataset_from_array()`` para instanciar tres datasets: uno para **entrenamiento**, otro para **validación** y otro para **prueba**.

Usaremos los siguientes valores de parámetro:

* ``sampling_rate = 6``: las observaciones se muestrearán en un punto de datos por hora: solo mantendremos un punto de datos de 6.
* ``sequence_length = 120``: las observaciones retrocederán 5 días (120 horas).
* ``delay = sampling_rate * (sequence_length + 24 - 1)``: el target de una secuencia será la temperatura 24 horas después del final de la secuencia.

In [6]:
from tensorflow import keras
sampling_rate = 6
sequence_length = 120
delay = sampling_rate * (sequence_length + 24 - 1)
batch_size = 256

train_dataset = keras.utils.timeseries_dataset_from_array(
    raw_data[:-delay],
    targets=temperature[delay:],
    sampling_rate=sampling_rate,
    sequence_length=sequence_length,
    shuffle=True,
    batch_size=batch_size,
    start_index=0,
    end_index=num_train_samples)

val_dataset = keras.utils.timeseries_dataset_from_array(
    raw_data[:-delay],
    targets=temperature[delay:],
    sampling_rate=sampling_rate,
    sequence_length=sequence_length,
    shuffle=True,
    batch_size=batch_size,
    start_index=num_train_samples,
    end_index=num_train_samples + num_val_samples)

test_dataset = keras.utils.timeseries_dataset_from_array(
    raw_data[:-delay],
    targets=temperature[delay:],
    sampling_rate=sampling_rate,
    sequence_length=sequence_length,
    shuffle=True,
    batch_size=batch_size,
    start_index=num_train_samples + num_val_samples)

Dado que el volumen de datos es demasiado grande como para poder trabajar con él sin emplear excesivo tiempo, de los 819 lotes de entrenamiento nos quedaremos con 50. De los lotes de validación con 25 y de los test otros 25.

In [7]:
train_dataset = train_dataset.take(50)
val_dataset = val_dataset.take(25)
test_dataset = test_dataset.take(25)

### Comenzando con un modelo de referencia con sentido común sin usar machine-learning



Esto servirá como línea base que se debe superar para demostrar la utilidad de los modelos de aprendizaje avanzado.

En este caso, **se puede suponer con cierta seguridad que las series de tiempo de temperatura son continuas**, así como **también periódicas con un período diario**. Por lo tanto, **un enfoque de sentido común es predecir siempre que la temperatura dentro de 24 horas será igual a la temperatura en este momento**.

Evaluemos este enfoque, utilizando la **métrica del error absoluto medio (MAE)**, definida de la siguiente manera:

**np.mean(np.abs(preds - targets))**



In [8]:
def evaluate_naive_method(dataset):
    total_abs_err = 0.
    samples_seen = 0
    for samples, targets in dataset:
        # La característica 'temperatura' está en la columna 1,
        # por lo que samples[:, -1, 1] es la última medición de
        # temperatura en la secuencia de entrada. Recordad que
        # normalizamos nuestras características, por lo que para
        # recuperar una temperatura en grados Celsius, debemos
        # desnormalizarla multiplicándola por la desviación
        # estándar y sumando nuevamente la media.
        preds = samples[:, -1, 1] * std[1] + mean[1]
        total_abs_err += np.sum(np.abs(preds - targets))
        samples_seen += samples.shape[0]
    return total_abs_err / samples_seen

print(f"Validation MAE: {evaluate_naive_method(val_dataset):.2f}")
print(f"Test MAE: {evaluate_naive_method(test_dataset):.2f}")

Validation MAE: 2.46
Test MAE: 2.60


Se ha logrado un MAE de validación de 2,46 y de prueba 2,60. Pese a que no es un resultado totalmente adecuado, si lo comparamos con el modelo base que se ha obtenido sin delimitar el conjunto de datos; apenas hay alguna diferencia en las centésimas.

## Modelo a ajustar.

Dado que utilizando parámetros casi aleatorios se obtiene un modelo de capas apiladas con unidades recurrentes cerradas (GRU) que ya supera la línea base, vamos a ver si, modificando algún hiperparámetro, logramos obtener un MAE más bajo.

Como hemos reducido notablemente la cantidad de datos, debemos disminuir el número de épocas para evitar que el modelo se sobreajuste.

Con los hiperparámetros que ya se daban se obtiene:

In [9]:
from tensorflow import keras
from tensorflow.keras import layers

In [10]:


inputs = keras.Input(shape=(sequence_length, raw_data.shape[-1]))
x = layers.GRU(32, recurrent_dropout=0.5, return_sequences=True)(inputs)
x = layers.GRU(32, recurrent_dropout=0.5)(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1)(x)
model = keras.Model(inputs, outputs)

callbacks = [
    keras.callbacks.ModelCheckpoint("jena_stacked_gru_dropout.keras",
                                    save_best_only=True)
]
model.compile(optimizer="rmsprop", loss="mse", metrics=["mae"])
history = model.fit(train_dataset,
                    epochs=20,
                    validation_data=val_dataset,
                    callbacks=callbacks)
model = keras.models.load_model("jena_stacked_gru_dropout.keras")
print(f"Test MAE: {model.evaluate(test_dataset)[1]:.2f}")

Epoch 1/20
50/50 [==============================] - 30s 510ms/step - loss: 91.5006 - mae: 7.8145 - val_loss: 52.6964 - val_mae: 5.6212
Epoch 2/20
50/50 [==============================] - 25s 491ms/step - loss: 48.1216 - mae: 5.2997 - val_loss: 39.7283 - val_mae: 4.7758
Epoch 3/20
50/50 [==============================] - 22s 434ms/step - loss: 38.0075 - mae: 4.6479 - val_loss: 29.9505 - val_mae: 4.0577
Epoch 4/20
50/50 [==============================] - 24s 472ms/step - loss: 30.6286 - mae: 4.1438 - val_loss: 23.0141 - val_mae: 3.5251
Epoch 5/20
50/50 [==============================] - 22s 443ms/step - loss: 25.2698 - mae: 3.7916 - val_loss: 18.5792 - val_mae: 3.1999
Epoch 6/20
50/50 [==============================] - 24s 479ms/step - loss: 21.4450 - mae: 3.5277 - val_loss: 15.2633 - val_mae: 2.9198
Epoch 7/20
50/50 [==============================] - 22s 431ms/step - loss: 19.4078 - mae: 3.3752 - val_loss: 13.4251 - val_mae: 2.7587
Epoch 8/20
50/50 [==============================] - 23s

Se obtiene un MAE de 2,54. Este es el valor que debemos mejorar.

## Aumentamos el número de unidades

Al aumentar el número de unidades, se aumenta la capacidad de apredizaje lo que puede llevar a aprender patrones complejos. Sin embargo, requiere mayor capacidad computacional o puede llevar a un sobreajuste del modelo.

### Aumentamos a 50

In [11]:

inputs = keras.Input(shape=(sequence_length, raw_data.shape[-1]))
x = layers.GRU(50, recurrent_dropout=0.5, return_sequences=True)(inputs)
x = layers.GRU(50, recurrent_dropout=0.5)(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1)(x)
model = keras.Model(inputs, outputs)

callbacks = [
    keras.callbacks.ModelCheckpoint("jena_stacked_gru_dropout.keras",
                                    save_best_only=True)
]
model.compile(optimizer="rmsprop", loss="mse", metrics=["mae"])
history = model.fit(train_dataset,
                    epochs=20,
                    validation_data=val_dataset,
                    callbacks=callbacks)
model = keras.models.load_model("jena_stacked_gru_dropout.keras")
print(f"Test MAE: {model.evaluate(test_dataset)[1]:.2f}")

Epoch 1/20
50/50 [==============================] - 37s 638ms/step - loss: 67.1910 - mae: 6.5345 - val_loss: 31.8657 - val_mae: 4.3140
Epoch 2/20
50/50 [==============================] - 32s 634ms/step - loss: 29.3321 - mae: 4.1328 - val_loss: 20.7911 - val_mae: 3.4122
Epoch 3/20
50/50 [==============================] - 32s 635ms/step - loss: 22.0187 - mae: 3.5833 - val_loss: 15.3592 - val_mae: 2.9410
Epoch 4/20
50/50 [==============================] - 31s 613ms/step - loss: 18.2992 - mae: 3.2969 - val_loss: 12.2456 - val_mae: 2.6578
Epoch 5/20
50/50 [==============================] - 38s 754ms/step - loss: 16.7297 - mae: 3.1738 - val_loss: 10.8098 - val_mae: 2.5035
Epoch 6/20
50/50 [==============================] - 37s 728ms/step - loss: 15.4670 - mae: 3.0629 - val_loss: 10.2150 - val_mae: 2.4661
Epoch 7/20
50/50 [==============================] - 32s 627ms/step - loss: 14.9480 - mae: 3.0046 - val_loss: 9.6268 - val_mae: 2.3886
Epoch 8/20
50/50 [==============================] - 32s 

Aumentando a 50 unidades hemos obtenido un MAE de 2,43. Este valor es mucho mejor que el inicial. Sin embargo, debemos comprobar si con menos unidades logramos resultados parecidos con menos coste computacional.

### Aumentamos a 40

In [12]:
inputs = keras.Input(shape=(sequence_length, raw_data.shape[-1]))
x = layers.GRU(40, recurrent_dropout=0.5, return_sequences=True)(inputs)
x = layers.GRU(40, recurrent_dropout=0.5)(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1)(x)
model = keras.Model(inputs, outputs)

callbacks = [
    keras.callbacks.ModelCheckpoint("jena_stacked_gru_dropout.keras",
                                    save_best_only=True)
]
model.compile(optimizer="rmsprop", loss="mse", metrics=["mae"])
history = model.fit(train_dataset,
                    epochs=20,
                    validation_data=val_dataset,
                    callbacks=callbacks)
model = keras.models.load_model("jena_stacked_gru_dropout.keras")
print(f"Test MAE: {model.evaluate(test_dataset)[1]:.2f}")

Epoch 1/20
50/50 [==============================] - 40s 662ms/step - loss: 82.1477 - mae: 7.3050 - val_loss: 44.8907 - val_mae: 5.1539
Epoch 2/20
50/50 [==============================] - 29s 586ms/step - loss: 40.7544 - mae: 4.8439 - val_loss: 31.7218 - val_mae: 4.2031
Epoch 3/20
50/50 [==============================] - 27s 529ms/step - loss: 30.9880 - mae: 4.2198 - val_loss: 22.9626 - val_mae: 3.5602
Epoch 4/20
50/50 [==============================] - 27s 540ms/step - loss: 24.4426 - mae: 3.7453 - val_loss: 17.9158 - val_mae: 3.1876
Epoch 5/20
50/50 [==============================] - 27s 545ms/step - loss: 20.1172 - mae: 3.4333 - val_loss: 14.0588 - val_mae: 2.8163
Epoch 6/20
50/50 [==============================] - 27s 537ms/step - loss: 17.9329 - mae: 3.2633 - val_loss: 11.9516 - val_mae: 2.6360
Epoch 7/20
50/50 [==============================] - 27s 540ms/step - loss: 16.7165 - mae: 3.1615 - val_loss: 10.7482 - val_mae: 2.4990
Epoch 8/20
50/50 [==============================] - 27s

Al disminuir el número de unidades ha aumentado el MAE por lo que 50 es el número óptimo de aumento.

## Disminuimos el número de unidades

Pese a que al aumentar se han obtenido mejores resultados, es necesario comprobar si con menos unidades los resultados mejoran o empeoran.



### Disminuimos a 16

In [13]:

inputs = keras.Input(shape=(sequence_length, raw_data.shape[-1]))
x = layers.GRU(16, recurrent_dropout=0.5, return_sequences=True)(inputs)
x = layers.GRU(16, recurrent_dropout=0.5)(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1)(x)
model = keras.Model(inputs, outputs)

callbacks = [
    keras.callbacks.ModelCheckpoint("jena_stacked_gru_dropout.keras",
                                    save_best_only=True)
]
model.compile(optimizer="rmsprop", loss="mse", metrics=["mae"])
history = model.fit(train_dataset,
                    epochs=20,
                    validation_data=val_dataset,
                    callbacks=callbacks)
model = keras.models.load_model("jena_stacked_gru_dropout.keras")
print(f"Test MAE: {model.evaluate(test_dataset)[1]:.2f}")

Epoch 1/20
50/50 [==============================] - 24s 379ms/step - loss: 120.0292 - mae: 9.0793 - val_loss: 90.6517 - val_mae: 7.8186
Epoch 2/20
50/50 [==============================] - 19s 374ms/step - loss: 78.8902 - mae: 6.9808 - val_loss: 69.7317 - val_mae: 6.5244
Epoch 3/20
50/50 [==============================] - 18s 348ms/step - loss: 66.0857 - mae: 6.2552 - val_loss: 59.0940 - val_mae: 5.9537
Epoch 4/20
50/50 [==============================] - 18s 356ms/step - loss: 57.6476 - mae: 5.7957 - val_loss: 52.3978 - val_mae: 5.5118
Epoch 5/20
50/50 [==============================] - 17s 345ms/step - loss: 51.7977 - mae: 5.4253 - val_loss: 43.9925 - val_mae: 4.9935
Epoch 6/20
50/50 [==============================] - 20s 396ms/step - loss: 45.4823 - mae: 5.0646 - val_loss: 38.5028 - val_mae: 4.6102
Epoch 7/20
50/50 [==============================] - 16s 313ms/step - loss: 40.6596 - mae: 4.7741 - val_loss: 33.7975 - val_mae: 4.3068
Epoch 8/20
50/50 [==============================] - 18

Al disminuir tanto las unidades, no se tienen suficientes como para realizar un proceso de aprendizaje adecuado. Por tanto, vamos a probar a no bajar demasiado ese número.

### Disminuimos a 25

In [14]:
inputs = keras.Input(shape=(sequence_length, raw_data.shape[-1]))
x = layers.GRU(25, recurrent_dropout=0.5, return_sequences=True)(inputs)
x = layers.GRU(25, recurrent_dropout=0.5)(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1)(x)
model = keras.Model(inputs, outputs)

callbacks = [
    keras.callbacks.ModelCheckpoint("jena_stacked_gru_dropout.keras",
                                    save_best_only=True)
]
model.compile(optimizer="rmsprop", loss="mse", metrics=["mae"])
history = model.fit(train_dataset,
                    epochs=20,
                    validation_data=val_dataset,
                    callbacks=callbacks)
model = keras.models.load_model("jena_stacked_gru_dropout.keras")
print(f"Test MAE: {model.evaluate(test_dataset)[1]:.2f}")

Epoch 1/20
50/50 [==============================] - 27s 441ms/step - loss: 92.7743 - mae: 7.9316 - val_loss: 53.1052 - val_mae: 5.6346
Epoch 2/20
50/50 [==============================] - 20s 401ms/step - loss: 50.4014 - mae: 5.4199 - val_loss: 42.2064 - val_mae: 4.9558
Epoch 3/20
50/50 [==============================] - 19s 385ms/step - loss: 41.3575 - mae: 4.8506 - val_loss: 34.0485 - val_mae: 4.3726
Epoch 4/20
50/50 [==============================] - 20s 390ms/step - loss: 34.7205 - mae: 4.4448 - val_loss: 27.6227 - val_mae: 3.8764
Epoch 5/20
50/50 [==============================] - 22s 429ms/step - loss: 29.4053 - mae: 4.0910 - val_loss: 22.2235 - val_mae: 3.4657
Epoch 6/20
50/50 [==============================] - 24s 468ms/step - loss: 25.5852 - mae: 3.8255 - val_loss: 18.7076 - val_mae: 3.1953
Epoch 7/20
50/50 [==============================] - 23s 456ms/step - loss: 23.2810 - mae: 3.6625 - val_loss: 16.1238 - val_mae: 2.9758
Epoch 8/20
50/50 [==============================] - 21s

Al obtener 2,55 de MAE podemos concluir que, aumentar el número de unidades, es más beneficioso para la predicción que disminuirlo.

## Modificamos el dropout

Otro hiperparámetro que podemos modificar es el dropout. Esto consiste en poner a cero las unidades de entrada de una capa de manera que se evite el sobreentrenamiento.
Normalmente el dropout no suele superar el 0.5. Además, en este caso al no contar con muchos datos poner más de la mitad de las entradas a cero seguro repercute negativamente en el resultado.

Por tanto, probaremos un dropout de 0,3.

In [16]:
inputs = keras.Input(shape=(sequence_length, raw_data.shape[-1]))
x = layers.GRU(50, recurrent_dropout=0.3, return_sequences=True)(inputs)
x = layers.GRU(50, recurrent_dropout=0.3)(x)
x = layers.Dropout(0.3)(x)
outputs = layers.Dense(1)(x)
model = keras.Model(inputs, outputs)

callbacks = [
    keras.callbacks.ModelCheckpoint("jena_stacked_gru_dropout.keras",
                                    save_best_only=True)
]
model.compile(optimizer="rmsprop", loss="mse", metrics=["mae"])
history = model.fit(train_dataset,
                    epochs=20,
                    validation_data=val_dataset,
                    callbacks=callbacks)
model = keras.models.load_model("jena_stacked_gru_dropout.keras")
print(f"Test MAE: {model.evaluate(test_dataset)[1]:.2f}")

Epoch 1/20
50/50 [==============================] - 36s 632ms/step - loss: 71.6423 - mae: 6.6610 - val_loss: 38.9475 - val_mae: 4.7640
Epoch 2/20
50/50 [==============================] - 39s 772ms/step - loss: 33.3525 - mae: 4.3727 - val_loss: 24.9508 - val_mae: 3.7261
Epoch 3/20
50/50 [==============================] - 32s 635ms/step - loss: 23.5832 - mae: 3.6728 - val_loss: 17.6417 - val_mae: 3.1133
Epoch 4/20
50/50 [==============================] - 32s 633ms/step - loss: 17.9625 - mae: 3.2337 - val_loss: 13.9116 - val_mae: 2.8126
Epoch 5/20
50/50 [==============================] - 32s 633ms/step - loss: 14.8556 - mae: 2.9719 - val_loss: 10.8978 - val_mae: 2.4953
Epoch 6/20
50/50 [==============================] - 32s 631ms/step - loss: 13.5993 - mae: 2.8689 - val_loss: 10.2583 - val_mae: 2.4361
Epoch 7/20
50/50 [==============================] - 39s 786ms/step - loss: 12.8520 - mae: 2.8040 - val_loss: 9.6096 - val_mae: 2.3795
Epoch 8/20
50/50 [==============================] - 37s 

El hecho de que no haya mejorado sustancialmente, quizás se puede explicar debido a que el posible sobreajuste que se puede provocar aumentando las unidades se contrasta con un dropout más alto. Por tanto, si considerásemos mejor opción menos unidades, se debería reducir el dropout y comprobar los resultados.

## Modificamos el learning_rate

Esta tasa indica cada cuánto se ajustan los pesos y se minimiza la pérdida. Si es demasiado pequeña puede quedarse en un mínimo local, mientras que si es demasiado grande nunca llegará al mínimo.

In [17]:
learning_rate = 0.001

optimizer = keras.optimizers.RMSprop(learning_rate=learning_rate)

inputs = keras.Input(shape=(sequence_length, raw_data.shape[-1]))
x = layers.GRU(50, recurrent_dropout=0.5, return_sequences=True)(inputs)
x = layers.GRU(50, recurrent_dropout=0.5)(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1)(x)
model = keras.Model(inputs, outputs)

callbacks = [
    keras.callbacks.ModelCheckpoint("jena_stacked_gru_dropout.keras",
                                    save_best_only=True)
]

model.compile(optimizer=optimizer, loss="mse", metrics=["mae"])
history = model.fit(train_dataset,
                    epochs=20,
                    validation_data=val_dataset,
                    callbacks=callbacks)
model = keras.models.load_model("jena_stacked_gru_dropout.keras")
print(f"Test MAE: {model.evaluate(test_dataset)[1]:.2f}")

Epoch 1/20
50/50 [==============================] - 45s 776ms/step - loss: 67.2501 - mae: 6.4761 - val_loss: 31.1041 - val_mae: 4.2465
Epoch 2/20
50/50 [==============================] - 38s 749ms/step - loss: 29.8619 - mae: 4.1765 - val_loss: 20.7982 - val_mae: 3.3950
Epoch 3/20
50/50 [==============================] - 34s 678ms/step - loss: 22.0159 - mae: 3.5786 - val_loss: 15.4117 - val_mae: 2.9628
Epoch 4/20
50/50 [==============================] - 32s 641ms/step - loss: 18.2107 - mae: 3.2912 - val_loss: 12.3884 - val_mae: 2.6732
Epoch 5/20
50/50 [==============================] - 32s 635ms/step - loss: 16.0633 - mae: 3.1123 - val_loss: 10.6599 - val_mae: 2.4794
Epoch 6/20
50/50 [==============================] - 32s 633ms/step - loss: 15.3348 - mae: 3.0375 - val_loss: 10.4421 - val_mae: 2.4766
Epoch 7/20
50/50 [==============================] - 32s 636ms/step - loss: 14.6524 - mae: 2.9917 - val_loss: 9.5730 - val_mae: 2.3829
Epoch 8/20
50/50 [==============================] - 32s 

Con respecto a la configuración que hemos determinado como última referencia, no supone mejora en el MAE.

Aumentemos este valor.

In [18]:
learning_rate = 0.01

optimizer = keras.optimizers.RMSprop(learning_rate=learning_rate)

inputs = keras.Input(shape=(sequence_length, raw_data.shape[-1]))
x = layers.GRU(50, recurrent_dropout=0.5, return_sequences=True)(inputs)
x = layers.GRU(50, recurrent_dropout=0.5)(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1)(x)
model = keras.Model(inputs, outputs)

callbacks = [
    keras.callbacks.ModelCheckpoint("jena_stacked_gru_dropout.keras",
                                    save_best_only=True)
]

model.compile(optimizer=optimizer, loss="mse", metrics=["mae"])
history = model.fit(train_dataset,
                    epochs=20,
                    validation_data=val_dataset,
                    callbacks=callbacks)
model = keras.models.load_model("jena_stacked_gru_dropout.keras")
print(f"Test MAE: {model.evaluate(test_dataset)[1]:.2f}")

Epoch 1/20
50/50 [==============================] - 47s 825ms/step - loss: 28.5115 - mae: 4.0017 - val_loss: 10.9267 - val_mae: 2.5347
Epoch 2/20
50/50 [==============================] - 32s 635ms/step - loss: 16.6862 - mae: 3.2106 - val_loss: 9.7124 - val_mae: 2.4168
Epoch 3/20
50/50 [==============================] - 32s 633ms/step - loss: 15.5318 - mae: 3.0952 - val_loss: 9.3883 - val_mae: 2.3626
Epoch 4/20
50/50 [==============================] - 32s 632ms/step - loss: 14.4453 - mae: 2.9863 - val_loss: 10.0897 - val_mae: 2.4803
Epoch 5/20
50/50 [==============================] - 32s 633ms/step - loss: 14.4307 - mae: 2.9671 - val_loss: 9.7842 - val_mae: 2.4215
Epoch 6/20
50/50 [==============================] - 32s 634ms/step - loss: 13.7979 - mae: 2.9006 - val_loss: 9.4905 - val_mae: 2.3971
Epoch 7/20
50/50 [==============================] - 32s 629ms/step - loss: 13.1911 - mae: 2.8417 - val_loss: 10.0710 - val_mae: 2.4931
Epoch 8/20
50/50 [==============================] - 32s 643

Como conclusión, se puede suponer que para este caso concreto un learning rate más pequeño favorece el aprendizaje del modelo.

Finalmente cambiaremos al optimizador Adam.

In [19]:
inputs = keras.Input(shape=(sequence_length, raw_data.shape[-1]))
x = layers.GRU(50, recurrent_dropout=0.5, return_sequences=True)(inputs)
x = layers.GRU(50, recurrent_dropout=0.5)(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1)(x)
model = keras.Model(inputs, outputs)

callbacks = [
    keras.callbacks.ModelCheckpoint("jena_stacked_gru_dropout.keras",
                                    save_best_only=True)
]
model.compile(optimizer="Adam", loss="mse", metrics=["mae"])
history = model.fit(train_dataset,
                    epochs=20,
                    validation_data=val_dataset,
                    callbacks=callbacks)
model = keras.models.load_model("jena_stacked_gru_dropout.keras")
print(f"Test MAE: {model.evaluate(test_dataset)[1]:.2f}")

Epoch 1/20
50/50 [==============================] - 38s 634ms/step - loss: 94.3170 - mae: 8.0826 - val_loss: 41.3988 - val_mae: 5.1043
Epoch 2/20
50/50 [==============================] - 31s 612ms/step - loss: 34.5124 - mae: 4.5172 - val_loss: 24.0830 - val_mae: 3.7081
Epoch 3/20
50/50 [==============================] - 31s 612ms/step - loss: 24.7687 - mae: 3.7950 - val_loss: 17.6214 - val_mae: 3.1550
Epoch 4/20
50/50 [==============================] - 32s 636ms/step - loss: 20.6992 - mae: 3.4741 - val_loss: 15.0315 - val_mae: 2.9098
Epoch 5/20
50/50 [==============================] - 34s 668ms/step - loss: 18.5440 - mae: 3.3111 - val_loss: 12.8139 - val_mae: 2.6962
Epoch 6/20
50/50 [==============================] - 32s 640ms/step - loss: 17.4906 - mae: 3.2317 - val_loss: 11.8063 - val_mae: 2.5964
Epoch 7/20
50/50 [==============================] - 32s 634ms/step - loss: 16.1047 - mae: 3.1098 - val_loss: 10.8817 - val_mae: 2.5062
Epoch 8/20
50/50 [==============================] - 32s

Cambiando el optimizador no se consigue mejorar el MAE.

## Capas Dense

In [21]:
inputs = keras.Input(shape=(sequence_length, raw_data.shape[-1]))
x = layers.GRU(50, recurrent_dropout=0.5, return_sequences=True)(inputs)
x = layers.GRU(50, recurrent_dropout=0.5)(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense (32)(x)
outputs = layers.Dense(1)(x)
model = keras.Model(inputs, outputs)

callbacks = [
    keras.callbacks.ModelCheckpoint("jena_stacked_gru_dropout.keras",
                                    save_best_only=True)
]
model.compile(optimizer="rmsprop", loss="mse", metrics=["mae"])
history = model.fit(train_dataset,
                    epochs=20,
                    validation_data=val_dataset,
                    callbacks=callbacks)
model = keras.models.load_model("jena_stacked_gru_dropout.keras")
print(f"Test MAE: {model.evaluate(test_dataset)[1]:.2f}")

Epoch 1/20
50/50 [==============================] - 36s 608ms/step - loss: 46.1398 - mae: 5.1591 - val_loss: 12.5734 - val_mae: 2.7757
Epoch 2/20
50/50 [==============================] - 32s 634ms/step - loss: 17.0247 - mae: 3.2350 - val_loss: 9.9500 - val_mae: 2.4534
Epoch 3/20
50/50 [==============================] - 31s 625ms/step - loss: 15.3097 - mae: 3.0651 - val_loss: 9.6416 - val_mae: 2.4172
Epoch 4/20
50/50 [==============================] - 32s 639ms/step - loss: 14.5791 - mae: 2.9915 - val_loss: 9.5526 - val_mae: 2.4045
Epoch 5/20
50/50 [==============================] - 32s 635ms/step - loss: 14.4736 - mae: 2.9786 - val_loss: 10.7394 - val_mae: 2.5779
Epoch 6/20
50/50 [==============================] - 32s 629ms/step - loss: 13.9149 - mae: 2.9262 - val_loss: 9.6307 - val_mae: 2.4202
Epoch 7/20
50/50 [==============================] - 36s 711ms/step - loss: 13.7197 - mae: 2.8968 - val_loss: 8.9703 - val_mae: 2.3373
Epoch 8/20
50/50 [==============================] - 33s 640m

Al añadir una capa Dense intermedia permitimos al modelo aprender patrones más específicos y conocer información extra que con respecto a las RNN. Como resultado, se obtiene el mejor MAE.